In [ ]:
!pip install transformers datasets seqeval -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_6

In [ ]:
import pandas as pd
import ast
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForTokenClassification, DataCollatorForTokenClassification, TrainingArguments, Trainer
from sklearn.model_selection import train_test_split
import numpy as np
from seqeval.metrics import classification_report

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from datasets import load_dataset

# Load dataset
raw_dataset = load_dataset("singh-aditya/MACCROBAT_biomedical_ner")  # Replace with actual dataset name

# Print dataset structure
print(raw_dataset)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/14.4k [00:00<?, ?B/s]

MACCROBAT_biomedical_ner.py:   0%|          | 0.00/5.75k [00:00<?, ?B/s]

The repository for singh-aditya/MACCROBAT_biomedical_ner contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/singh-aditya/MACCROBAT_biomedical_ner.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['full_text', 'ner_info', 'tokens', 'ner_labels'],
        num_rows: 200
    })
})


In [ ]:
# Extract labels from the dataset features
features = raw_dataset["train"].features
label_list = features["ner_labels"].feature.names
label2id = {label: idx for idx, label in enumerate(label_list)}
id2label = {idx: label for label, idx in label2id.items()}

In [ ]:
model_name = "bert-base-uncased"

# Tokenization
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"],
        is_split_into_words=True,
        truncation=True,
        padding=True
    )
    all_labels = []
    for i, word_ids in enumerate(tokenized_inputs.word_ids(batch_index=i) for i in range(len(examples["tokens"]))):
        labels = []
        prev_word_id = None
        for word_id in word_ids:
            if word_id is None:
                labels.append(-100)
            elif word_id != prev_word_id:
                labels.append(examples["ner_labels"][i][word_id])
            else:
                labels.append(-100)
            prev_word_id = word_id
        all_labels.append(labels)
    tokenized_inputs["labels"] = all_labels
    return tokenized_inputs

In [ ]:
# Split dataset
train_test = raw_dataset["train"].train_test_split(test_size=0.2)
train_dataset = train_test["train"].map(tokenize_and_align_labels, batched=True)
train_dataset = train_dataset.remove_columns(raw_dataset["train"].column_names)
test_dataset = train_test["test"].map(tokenize_and_align_labels, batched=True)
test_dataset = test_dataset.remove_columns(raw_dataset["train"].column_names)


print("Train samples:", len(train_dataset))
print("Test samples:", len(test_dataset))

Map:   0%|          | 0/160 [00:00<?, ? examples/s]

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

Train samples: 160
Test samples: 40


In [ ]:
# Model setup
model = AutoModelForTokenClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=len(label2id),
    id2label=id2label,
    label2id=label2id
)

data_collator = DataCollatorForTokenClassification(tokenizer)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# # Metrics
# def compute_metrics(p):
#     predictions, labels = p
#     predictions = np.argmax(predictions, axis=2)
#     true_predictions = [
#         [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
#         for prediction, label in zip(predictions, labels)
#     ]
#     true_labels = [
#         [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
#         for prediction, label in zip(predictions, labels)
#     ]
#     report = classification_report(true_labels, true_predictions, output_dict=True)
#     return {
#         "precision": report["micro avg"]["precision"],
#         "recall": report["micro avg"]["recall"],
#         "f1": report["micro avg"]["f1-score"]
#     }

In [ ]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)
    true_predictions = []
    true_labels = []
    correct = 0
    total = 0

    for prediction, label in zip(predictions, labels):
        current_preds = []
        current_labels = []
        for p, l in zip(prediction, label):
            if l != -100:
                current_preds.append(label_list[p])
                current_labels.append(label_list[l])
                if p == l:
                    correct += 1
                total += 1
        true_predictions.append(current_preds)
        true_labels.append(current_labels)

    report = classification_report(true_labels, true_predictions, output_dict=True)
    accuracy = correct / total if total > 0 else 0.0

    return {
        "precision": report["micro avg"]["precision"],
        "recall": report["micro avg"]["recall"],
        "f1": report["micro avg"]["f1-score"],
        "accuracy": accuracy
    }

In [ ]:
# Training
training_args = TrainingArguments(
    output_dir="bert-med-ner",
    evaluation_strategy="epoch",
    metric_for_best_model="f1",
    load_best_model_at_end=True,
    learning_rate=1e-5,  # lowered learning rate, explored different LRs and retained this as this resulted in best of all.
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=20,  # increased from 10
    weight_decay=0.01,
    logging_dir="logs",
    logging_steps=10,
    save_strategy="epoch",
    remove_unused_columns=False
)

from transformers import EarlyStoppingCallback

# Trainer
trainer = Trainer(
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-12-0baaf8f0a150>:21: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: prithvi-elancherran (prithvi-elancherran-san-jose-state-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,2.850900,2.501451,0.000000,0.000000,0.000000,0.442242
2,2.050600,1.832582,0.221076,0.166574,0.189994,0.532673
3,1.684100,1.526582,0.339678,0.322043,0.330626,0.617374
4,1.462700,1.332623,0.458713,0.447252,0.452910,0.686126
5,1.318600,1.172788,0.523755,0.511105,0.517353,0.722515
6,1.165900,1.057823,0.569149,0.564409,0.566769,0.751936
7,1.089400,0.970398,0.598993,0.594670,0.596824,0.768194
8,0.984900,0.905342,0.612074,0.607996,0.610028,0.775317
9,0.917700,0.859925,0.621772,0.621599,0.621685,0.781511
10,0.856800,0.824507,0.633306,0.632426,0.632866,0.787860


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are i

TrainOutput(global_step=400, training_loss=1.1326948916912079, metrics={'train_runtime': 926.9171, 'train_samples_per_second': 3.452, 'train_steps_per_second': 0.432, 'total_flos': 836761947340800.0, 'train_loss': 1.1326948916912079, 'epoch': 20.0})

In [ ]:
trainer.evaluate()

/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'eval_loss': 0.6893399357795715,
 'eval_precision': 0.7089241034195163,
 'eval_recall': 0.7079400333148251,
 'eval_f1': 0.7084317266286985,
 'eval_accuracy': 0.8301331681635181,
 'eval_runtime': 1.363,
 'eval_samples_per_second': 29.348,
 'eval_steps_per_second': 3.668,
 'epoch': 20.0}

In [ ]:
# Saving fine-tuned model and tokenizer to Hugging Face Hub
model.push_to_hub("bert-med-ner", private=True)
tokenizer.push_to_hub("bert-med-ner", private=True)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/PrithviElancherran/bert-med-ner/commit/069b2dbb224d73bf16da2ea34a5a1af936bbcfb0', commit_message='Upload tokenizer', commit_description='', oid='069b2dbb224d73bf16da2ea34a5a1af936bbcfb0', pr_url=None, repo_url=RepoUrl('https://huggingface.co/PrithviElancherran/bert-med-ner', endpoint='https://huggingface.co', repo_type='model', repo_id='PrithviElancherran/bert-med-ner'), pr_revision=None, pr_num=None)

In [ ]:
# Saving fine-tuned model and tokenizer locally
model.save_pretrained("saved-bert-med-ner")
tokenizer.save_pretrained("saved-bert-med-ner")

('saved-bert-med-ner/tokenizer_config.json',
 'saved-bert-med-ner/special_tokens_map.json',
 'saved-bert-med-ner/vocab.txt',
 'saved-bert-med-ner/added_tokens.json',
 'saved-bert-med-ner/tokenizer.json')

In [ ]:
# saving model folder for download
import shutil
shutil.make_archive("bert-med-ner-model", 'zip', "saved-bert-med-ner")

'/content/bert-med-ner-model.zip'

In [ ]:
from transformers import AutoModelForTokenClassification, AutoTokenizer

model = AutoModelForTokenClassification.from_pretrained("path/to/unzipped-folder")
tokenizer = AutoTokenizer.from_pretrained("path/to/unzipped-folder")


### **This block is For testing Finetuned BERT output on an OOV clinical note**

In [ ]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 68.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.6 MB/s eta 0:00:00


In [ ]:
# Gradio Inference UI
import gradio as gr
import torch

def ner_inference(text):
    # Tokenize input
    tokens = tokenizer.tokenize(text)
    inputs = tokenizer(text, return_tensors="pt")
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=2)[0].cpu().numpy()
    tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])

    # Map predictions to labels
    labeled_tokens = []
    for token, pred in zip(tokens, predictions):
        label = id2label.get(pred, "O")
        labeled_tokens.append(f"{token} [{label}]")
    return " ".join(labeled_tokens)

demo = gr.Interface(
    fn=ner_inference,
    inputs=gr.Textbox(lines=5, placeholder="Enter clinical/medical text here..."),
    outputs="text",
    title="Medical NER - BERT",
    description="Enter medical sentences to extract entities using the fine-tuned BERT model."
)

demo.launch(debug=True)


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://6db861e96582cce1ae.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://6db861e96582cce1ae.gradio.live
